In [279]:
import pandas as pd
import numpy as np
import math
from scipy.stats import norm
import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [280]:
def find_upper_bound(mu, std, alpha=0.95):
    z_value = norm.ppf(alpha)
    return mu + std * z_value

In [281]:
gpu_conv_value = 0.00021648585046
gpu_no_norm_conv_value = 6958.933333333333333
per_unit_energy_cons = 1
target_qos = ["90", "91", "92", "93", "94", "95", "96", "97", "98", "99"]

In [282]:
original_dataset = pd.read_csv("../saved_data/ali20/ali20_g.csv")
scaler = MinMaxScaler()
factor=0.8
scaler.fit(original_dataset.avggpu.iloc[:int(original_dataset.shape[0]*factor)].values.reshape(-1,1))
original_dataset.describe()

,Unnamed: 0,time,avggpu,avggpumem
count,14358.000000,1.435800e+04,1.435800e+04,1.435800e+04
mean,8928.500000,1.596240e+09,2.062431e+07,3.858848e+06
std,4144.941918,1.243483e+06,4.407054e+06,9.409406e+05
min,1750.000000,1.594087e+09,6.682396e+06,1.223004e+06
25%,5339.250000,1.595163e+09,1.780858e+07,3.268815e+06
50%,8928.500000,1.596240e+09,2.088287e+07,3.882400e+06
75%,12517.750000,1.597317e+09,2.357788e+07,4.509566e+06
max,16107.000000,1.598394e+09,3.368392e+07,6.902691e+06


In [283]:
hbnn_results = pd.read_csv("output_HBNN-ali20_g-gpu-w288-h2.csv")
hbnn_results.drop(labels=["Unnamed: 0"], inplace=True, axis=1)
hbnn_results["model"] = ["HBNN" for i in range(len(hbnn_results))]
hbnn_results.rename(columns={"avggpu": "pred_norm_gpu", "std": "pred_std", "labels": "true_norm_gpu"}, inplace=True)
first_column = hbnn_results.pop('model')
hbnn_results.insert(0, 'model', first_column)
hbnn_results["true_gpu"] = original_dataset["avggpu"].values[-2582:]
hbnn_results["true_n_gpu"] = hbnn_results.apply(lambda row: math.ceil(row["true_gpu"]/gpu_no_norm_conv_value), axis=1)
# hbnn_results.drop(columns=['true_norm_gpu'], inplace=True)
hbnn_results["pred_gpu"] = scaler.inverse_transform(hbnn_results["pred_norm_gpu"].values.reshape(-1,1))
for target in target_qos:
    alpa = float(int(target)/100)
    hbnn_results[f"ub_{target}_norm"] = hbnn_results.apply(lambda row: find_upper_bound(row["pred_norm_gpu"], row["pred_std"], alpha=alpa), axis=1)
    hbnn_results[f"ub_{target}"] = scaler.inverse_transform(hbnn_results[f"ub_{target}_norm"].values.reshape(-1,1))
    hbnn_results[f"pred_n_gpu_{target}"] = hbnn_results.apply(lambda row: math.ceil(row[f"ub_{target}"]/gpu_no_norm_conv_value), axis=1)
hbnn_results.to_csv("scenarios/hbnn_results.csv")
hbnn_results.describe()

,pred_norm_gpu,pred_std,true_norm_gpu,true_gpu,true_n_gpu,pred_gpu,ub_90_norm,ub_90,pred_n_gpu_90,ub_91_norm,...,pred_n_gpu_96,ub_97_norm,ub_97,pred_n_gpu_97,ub_98_norm,ub_98,pred_n_gpu_98,ub_99_norm,ub_99,pred_n_gpu_99
count,2582.000000,2582.000000,2582.000000,2.582000e+03,2582.000000,2.582000e+03,2582.000000,2.582000e+03,2582.000000,2582.000000,...,2582.000000,2582.000000,2.582000e+03,2582.000000,2582.000000,2.582000e+03,2582.000000,2582.000000,2.582000e+03,2582.000000
mean,0.357420,0.049786,0.328949,1.822752e+07,2619.806352,1.833324e+07,0.421223,1.970464e+07,2832.052672,0.424171,...,2904.207204,0.451057,2.034590e+07,2924.202556,0.459668,2.053098e+07,2950.803641,0.473239,2.082269e+07,2992.729280
std,0.187234,0.007063,0.232520,5.355664e+06,769.605537,4.024433e+06,0.188831,4.058763e+06,583.245436,0.188915,...,585.367004,0.189721,4.077902e+06,586.005779,0.189995,4.083786e+06,586.841574,0.190441,4.093385e+06,588.223474
min,0.109442,0.016079,-0.172291,6.682396e+06,961.000000,1.300316e+07,0.167206,1.424474e+07,2047.000000,0.169874,...,2113.000000,0.194216,1.482530e+07,2131.000000,0.202011,1.499286e+07,2155.000000,0.214298,1.525696e+07,2193.000000
25%,0.143482,0.046260,0.106536,1.310465e+07,1884.000000,1.373481e+07,0.210610,1.517768e+07,2181.500000,0.213763,...,2260.000000,0.242669,1.586675e+07,2280.500000,0.251706,1.606100e+07,2308.250000,0.265869,1.636542e+07,2352.000000
50%,0.372455,0.049210,0.362540,1.900122e+07,2731.000000,1.865642e+07,0.433462,1.996770e+07,2869.500000,0.436478,...,2933.000000,0.460769,2.055465e+07,2954.500000,0.468404,2.071875e+07,2977.500000,0.481697,2.100448e+07,3019.000000
75%,0.516378,0.052960,0.519670,2.262042e+07,3251.000000,2.174992e+07,0.581171,2.314260e+07,3325.750000,0.584130,...,3398.000000,0.611112,2.378614e+07,3419.000000,0.619926,2.397560e+07,3445.750000,0.633787,2.427352e+07,3488.500000
max,0.825197,0.110551,0.915283,3.173261e+07,4560.000000,2.838773e+07,0.898194,2.995674e+07,4305.000000,0.901566,...,4388.000000,0.932327,3.069040e+07,4411.000000,0.942178,3.090215e+07,4441.000000,0.957705,3.123589e+07,4489.000000


In [284]:
monte_results = pd.read_csv("output_MCDLSTM-ali20_g-gpu-w288-h2.csv")
monte_results.drop(labels=["Unnamed: 0"], inplace=True, axis=1)
monte_results["model"] = ["MCD" for i in range(len(monte_results))]
monte_results.rename(columns={"avggpu": "pred_norm_gpu", "std": "pred_std", "labels": "true_norm_gpu"}, inplace=True)
first_column = monte_results.pop('model')
monte_results.insert(0, 'model', first_column)
monte_results["true_gpu"] = original_dataset["avggpu"].values[-2582:]
monte_results["true_n_gpu"] = monte_results.apply(lambda row: math.ceil(row["true_gpu"]/gpu_no_norm_conv_value), axis=1)
# monte_results.drop(columns=['true_norm_gpu'], inplace=True)
monte_results["pred_gpu"] = scaler.inverse_transform(monte_results["pred_norm_gpu"].values.reshape(-1,1))

for target in target_qos:
    alpa = float(int(target)/100)
    monte_results[f"ub_{target}_norm"] = monte_results.apply(lambda row: find_upper_bound(row["pred_norm_gpu"], row["pred_std"], alpha=alpa), axis=1)
    monte_results[f"ub_{target}"] = scaler.inverse_transform(monte_results[f"ub_{target}_norm"].values.reshape(-1,1))
    monte_results[f"pred_n_gpu_{target}"] = monte_results.apply(lambda row: math.ceil(row[f"ub_{target}"]/gpu_no_norm_conv_value), axis=1)
monte_results.to_csv("scenarios/monte_results.csv")
monte_results.describe()

,pred_norm_gpu,pred_std,true_norm_gpu,true_gpu,true_n_gpu,pred_gpu,ub_90_norm,ub_90,pred_n_gpu_90,ub_91_norm,...,pred_n_gpu_96,ub_97_norm,ub_97,pred_n_gpu_97,ub_98_norm,ub_98,pred_n_gpu_98,ub_99_norm,ub_99,pred_n_gpu_99
count,2582.000000,2.582000e+03,2582.000000,2.582000e+03,2582.000000,2.582000e+03,2582.000000,2.582000e+03,2582.000000,2582.000000,...,2582.000000,2582.000000,2.582000e+03,2582.000000,2582.000000,2.582000e+03,2582.000000,2582.000000,2.582000e+03,2582.000000
mean,0.337572,6.597421e-08,0.328949,1.822752e+07,2619.806352,1.790663e+07,0.337572,1.790664e+07,2573.692874,0.337572,...,2573.692874,0.337572,1.790664e+07,2573.692874,0.337573,1.790664e+07,2573.693261,0.337573,1.790664e+07,2573.693261
std,0.214559,6.558250e-08,0.232520,5.355664e+06,769.605537,4.611775e+06,0.214559,4.611776e+06,662.714245,0.214559,...,662.714245,0.214559,4.611777e+06,662.714245,0.214559,4.611777e+06,662.713676,0.214559,4.611777e+06,662.713676
min,-0.039172,0.000000e+00,-0.172291,6.682396e+06,961.000000,9.808823e+06,-0.039172,9.808824e+06,1410.000000,-0.039172,...,1410.000000,-0.039172,9.808824e+06,1410.000000,-0.039172,9.808824e+06,1410.000000,-0.039172,9.808824e+06,1410.000000
25%,0.125868,1.117587e-08,0.106536,1.310465e+07,1884.000000,1.335623e+07,0.125868,1.335623e+07,1919.500000,0.125868,...,1919.500000,0.125868,1.335623e+07,1919.500000,0.125868,1.335623e+07,1919.500000,0.125868,1.335623e+07,1919.500000
50%,0.367512,5.960465e-08,0.362540,1.900122e+07,2731.000000,1.855016e+07,0.367512,1.855016e+07,2666.000000,0.367512,...,2666.000000,0.367512,1.855016e+07,2666.000000,0.367512,1.855016e+07,2666.000000,0.367512,1.855016e+07,2666.000000
75%,0.512203,8.940697e-08,0.519670,2.262042e+07,3251.000000,2.166017e+07,0.512203,2.166018e+07,3112.750000,0.512203,...,3112.750000,0.512203,2.166018e+07,3112.750000,0.512203,2.166018e+07,3112.750000,0.512203,2.166018e+07,3112.750000
max,0.893295,2.980232e-07,0.915283,3.173261e+07,4560.000000,2.985144e+07,0.893295,2.985145e+07,4290.000000,0.893295,...,4290.000000,0.893295,2.985145e+07,4290.000000,0.893295,2.985145e+07,4290.000000,0.893295,2.985145e+07,4290.000000


In [285]:
flbnn_results = pd.read_csv("output_FLBNN-ali20_g-gpu-w288-h2.csv")
flbnn_results.drop(labels=["Unnamed: 0"], inplace=True, axis=1)
flbnn_results["model"] = ["HBNN++" for i in range(len(flbnn_results))]
flbnn_results.rename(columns={"avggpu": "pred_norm_gpu", "std": "pred_std", "labels": "true_norm_gpu"}, inplace=True)
first_column = flbnn_results.pop('model')
flbnn_results.insert(0, 'model', first_column)
flbnn_results["true_gpu"] = original_dataset["avggpu"].values[-2582:]
flbnn_results["true_n_gpu"] = flbnn_results.apply(lambda row: math.ceil(row["true_gpu"]/gpu_no_norm_conv_value), axis=1)
# flbnn_results.drop(columns=['true_norm_gpu'], inplace=True)
flbnn_results["pred_gpu"] = scaler.inverse_transform(flbnn_results["pred_norm_gpu"].values.reshape(-1,1))

for target in target_qos:
    alpa = float(int(target)/100)
    flbnn_results[f"ub_{target}_norm"] = flbnn_results.apply(lambda row: find_upper_bound(row["pred_norm_gpu"], row["pred_std"], alpha=alpa), axis=1)
    flbnn_results[f"ub_{target}"] = scaler.inverse_transform(flbnn_results[f"ub_{target}_norm"].values.reshape(-1,1))
    flbnn_results[f"pred_n_gpu_{target}"] = flbnn_results.apply(lambda row: math.ceil(row[f"ub_{target}"]/gpu_no_norm_conv_value), axis=1)
flbnn_results.to_csv("scenarios/flbnn_results.csv")
flbnn_results.describe()

,pred_norm_gpu,pred_std,true_norm_gpu,true_gpu,true_n_gpu,pred_gpu,ub_90_norm,ub_90,pred_n_gpu_90,ub_91_norm,...,pred_n_gpu_96,ub_97_norm,ub_97,pred_n_gpu_97,ub_98_norm,ub_98,pred_n_gpu_98,ub_99_norm,ub_99,pred_n_gpu_99
count,2582.000000,2582.000000,2582.000000,2.582000e+03,2582.000000,2.582000e+03,2582.000000,2.582000e+03,2582.000000,2582.000000,...,2582.000000,2582.000000,2.582000e+03,2582.000000,2582.000000,2.582000e+03,2582.000000,2582.000000,2.582000e+03,2582.000000
mean,0.362692,0.066016,0.328949,1.822752e+07,2619.806352,1.844655e+07,0.447295,2.026503e+07,2912.587529,0.451203,...,3008.252905,0.486855,2.111534e+07,3034.780790,0.498273,2.136076e+07,3070.043377,0.516269,2.174757e+07,3125.620837
std,0.160845,0.010062,0.232520,5.355664e+06,769.605537,3.457235e+06,0.172483,3.707383e+06,532.743167,0.173025,...,546.049709,0.177984,3.825629e+06,549.741435,0.179578,3.859893e+06,554.664675,0.182096,3.914014e+06,562.443335
min,0.128860,0.043463,-0.172291,6.682396e+06,961.000000,1.342052e+07,0.198997,1.492808e+07,2146.000000,0.202238,...,2225.000000,0.231793,1.563300e+07,2247.000000,0.241259,1.583645e+07,2276.000000,0.256178,1.615713e+07,2322.000000
25%,0.163772,0.053720,0.106536,1.310465e+07,1884.000000,1.417095e+07,0.230023,1.559494e+07,2241.250000,0.233086,...,2316.250000,0.261072,1.626231e+07,2337.250000,0.270133,1.645707e+07,2365.250000,0.284297,1.676152e+07,2409.000000
50%,0.404047,0.070484,0.362540,1.900122e+07,2731.000000,1.933546e+07,0.496730,2.132760e+07,3065.000000,0.500986,...,3169.500000,0.539615,2.224938e+07,3198.000000,0.551925,2.251397e+07,3236.000000,0.571510,2.293493e+07,3296.500000
75%,0.510996,0.074996,0.519670,2.262042e+07,3251.000000,2.163423e+07,0.608714,2.373460e+07,3411.000000,0.613219,...,3522.000000,0.654395,2.471648e+07,3552.000000,0.667527,2.499874e+07,3593.000000,0.688476,2.544902e+07,3657.750000
max,0.619214,0.090434,0.915283,3.173261e+07,4560.000000,2.396029e+07,0.699979,2.569626e+07,3693.000000,0.703710,...,3784.000000,0.737744,2.650799e+07,3810.000000,0.748643,2.674227e+07,3843.000000,0.765823,2.711153e+07,3896.000000


In [286]:
lstmq_results = pd.read_csv("LSTMQ/output_LSTMQ-0.95-ali20_g-gpu-w288-h2.csv")
lstmq_results.drop(labels=["Unnamed: 0"], inplace=True, axis=1)
lstmq_results["model"] = ["LSTMQ" for i in range(len(lstmq_results))]
lstmq_results.rename(columns={"avggpu": "pred_norm_gpu", "std": "pred_std", "labels": "true_norm_gpu"}, inplace=True)
first_column = lstmq_results.pop('model')
lstmq_results.insert(0, 'model', first_column)
lstmq_results["true_gpu"] = original_dataset["avggpu"].values[-2582:]
lstmq_results["true_n_gpu"] = lstmq_results.apply(lambda row: math.ceil(row["true_gpu"]/gpu_no_norm_conv_value), axis=1)
# lstmq_results.drop(columns=['true_norm_gpu'], inplace=True)

for target in target_qos:
    alpa = float(int(target)/100)
    target_results = pd.read_csv(f"LSTMQ/output_LSTMQ-{alpa}-ali20_g-gpu-w288-h2.csv")
    lstmq_results[f"ub_{target}_norm"] = target_results.apply(lambda row: find_upper_bound(row["avggpu"], row["std"], alpha=alpa), axis=1)
    lstmq_results[f"ub_{target}"] = scaler.inverse_transform(lstmq_results[f"ub_{target}_norm"].values.reshape(-1,1))
    lstmq_results[f"pred_n_gpu_{target}"] = lstmq_results.apply(lambda row: math.ceil(row[f"ub_{target}"]/gpu_no_norm_conv_value), axis=1)
lstmq_results.to_csv("scenarios/lstmq_results.csv")
lstmq_results.describe()

,pred_norm_gpu,pred_std,true_norm_gpu,true_gpu,true_n_gpu,ub_90_norm,ub_90,pred_n_gpu_90,ub_91_norm,ub_91,...,pred_n_gpu_96,ub_97_norm,ub_97,pred_n_gpu_97,ub_98_norm,ub_98,pred_n_gpu_98,ub_99_norm,ub_99,pred_n_gpu_99
count,2582.000000,2582.0,2582.000000,2.582000e+03,2582.000000,2582.000000,2.582000e+03,2582.000000,2582.000000,2.582000e+03,...,2582.000000,2582.000000,2.582000e+03,2582.000000,2582.000000,2.582000e+03,2582.000000,2582.000000,2.582000e+03,2582.000000
mean,0.401227,0.0,0.328949,1.822752e+07,2619.806352,0.365301,1.850264e+07,2659.324555,0.379118,1.879962e+07,...,2810.230054,0.410383,1.947165e+07,2798.585980,0.405689,1.937074e+07,2784.068552,0.504672,2.149830e+07,3089.815647
std,0.227837,0.0,0.232520,5.355664e+06,769.605537,0.227710,4.894433e+06,703.334291,0.229850,4.940434e+06,...,678.332355,0.200764,4.315268e+06,620.102623,0.253810,5.455447e+06,783.955297,0.203512,4.374329e+06,628.588877
min,0.005820,0.0,-0.172291,6.682396e+06,961.000000,-0.044874,9.686262e+06,1392.000000,-0.011003,1.041430e+07,...,1711.000000,0.065909,1.206745e+07,1735.000000,-0.193060,6.501125e+06,935.000000,0.136283,1.358009e+07,1952.000000
25%,0.190937,0.0,0.106536,1.310465e+07,1884.000000,0.147834,1.382836e+07,1988.000000,0.155301,1.398885e+07,...,2146.500000,0.223268,1.544976e+07,2220.250000,0.240581,1.582188e+07,2274.250000,0.317426,1.747360e+07,2511.500000
50%,0.431730,0.0,0.362540,1.900122e+07,2731.000000,0.398979,1.922653e+07,2763.000000,0.411144,1.948800e+07,...,2888.000000,0.428514,1.986135e+07,2854.500000,0.454264,2.041482e+07,2934.000000,0.525413,2.194413e+07,3154.000000
75%,0.588165,0.0,0.519670,2.262042e+07,3251.000000,0.551565,2.250622e+07,3234.750000,0.568246,2.286477e+07,...,3370.500000,0.574925,2.300833e+07,3307.000000,0.599153,2.352909e+07,3382.000000,0.663141,2.490447e+07,3579.000000
max,0.982749,0.0,0.915283,3.173261e+07,4560.000000,0.910586,3.022311e+07,4344.000000,0.961678,3.132128e+07,...,4444.000000,0.927526,3.058721e+07,4396.000000,0.991726,3.196713e+07,4594.000000,1.035353,3.290486e+07,4729.000000


In [287]:
perc_upper_bound = 0.05
lstm_results = pd.read_csv("output_LSTM-ali20_g-gpu-w288-h2.csv")
lstm_results.drop(labels=["Unnamed: 0"], inplace=True, axis=1)
lstm_results["model"] = ["LSTMQ" for i in range(len(lstm_results))]
lstm_results.rename(columns={"avggpu": "pred_norm_gpu", "std": "pred_std", "labels": "true_norm_gpu"}, inplace=True)
first_column = lstm_results.pop('model')
lstm_results.insert(0, 'model', first_column)
lstm_results["true_gpu"] = original_dataset["avggpu"].values[-2582:]
lstm_results["true_n_gpu"] = lstm_results.apply(lambda row: math.ceil(row["true_gpu"]/gpu_no_norm_conv_value), axis=1)
# lstm_results.drop(columns=['true_norm_gpu'], inplace=True)
lstm_results["pred_gpu"] = scaler.inverse_transform(lstm_results["pred_norm_gpu"].values.reshape(-1,1))

for target in target_qos:
    alpa = float(int(target)/100)
    lstm_results[f"ub_{target}_norm"] = lstm_results["pred_norm_gpu"].values
    lstm_results[f"ub_{target}_norm"] = [value+(perc_upper_bound*value) for value in lstm_results[f"ub_{target}_norm"].values]
    lstm_results[f"ub_{target}"] = scaler.inverse_transform(lstm_results[f"ub_{target}_norm"].values.reshape(-1,1))
    lstm_results[f"pred_n_gpu_{target}"] = lstm_results.apply(lambda row: math.ceil(row[f"ub_{target}"]/gpu_no_norm_conv_value), axis=1)
lstm_results.to_csv("scenarios/lstm_results.csv")
lstm_results.head(2)

,model,pred_norm_gpu,true_norm_gpu,true_gpu,true_n_gpu,pred_gpu,ub_90_norm,ub_90,pred_n_gpu_90,ub_91_norm,...,pred_n_gpu_96,ub_97_norm,ub_97,pred_n_gpu_97,ub_98_norm,ub_98,pred_n_gpu_98,ub_99_norm,ub_99,pred_n_gpu_99
0,LSTMQ,0.835576,0.804885,2.918980e+07,4195,2.861081e+07,0.877354,2.950881e+07,4241,0.877354,...,4241,0.877354,2.950881e+07,4241,0.877354,2.950881e+07,4241,0.877354,2.950881e+07,4241
1,LSTMQ,0.822173,0.814360,2.940805e+07,4226,2.832273e+07,0.863281,2.920633e+07,4197,0.863281,...,4197,0.863281,2.920633e+07,4197,0.863281,2.920633e+07,4197,0.863281,2.920633e+07,4197


<h2>Accuracy</h2>

In [248]:
def calculate_accuracy(df_results, tg):
    mse_value = mean_squared_error(df_results[f"ub_{tg}_norm"].values, df_results["true_norm_gpu"].values)
    mae_value = mean_absolute_error(df_results[f"ub_{tg}_norm"].values, df_results["true_norm_gpu"].values)
    return mse_value, mae_value

def calculate_success(df_results, tg):
    sr_value = sum(df_results.ub_95>df_results.true_gpu)/df_results.shape[0]
    up_value = sum((df_results.ub_95<df_results.true_gpu)*(df_results.true_gpu-df_results.ub_95))/sum(df_results.true_gpu)
    op_value = sum((df_results.ub_95>df_results.true_gpu)*(df_results.ub_95-df_results.true_gpu))/sum(df_results.true_gpu)
    return sr_value, up_value, op_value

In [249]:
models = {"MCDLSTM": monte_results,
          "HBNN": hbnn_results,
          "FLBNN": flbnn_results,
          "LSTMQ": lstmq_results,
          "LSTM": lstm_results
          }

In [250]:
mse_column = []
mae_column = []
model_column = []
target_column = []
for model in models:
    print(model)
    for target in target_qos:
        if model == "LSTMQ":
            a, b = "NA", "NA"
        else:
            a, b = calculate_accuracy(models[model], target)
        mse_column.append(a)
        mae_column.append(b)
        model_column.append(model)
        target_column.append(target)
d = {"model": model_column,
     "qos": target_column,
     "mse": mse_column,
     "mae": mae_column
     }
df_accuracy = pd.DataFrame(data=d)

MCDLSTM
HBNN
FLBNN
LSTMQ
LSTM


In [251]:
df_accuracy.head(1)

,model,qos,mse,mae
0,MCDLSTM,90,0.002068,0.033906


In [252]:
sr_column = []
up_column = []
op_column = []
model_column = []
target_column = []
for model in models:
    print(model)
    for target in target_qos:
        a, b, c = calculate_success(models[model], target)
        sr_column.append(a)
        up_column.append(b)
        op_column.append(c)
        model_column.append(model)
        target_column.append(target)
d = {"model": model_column,
     "qos": target_column,
     "SR": sr_column,
     "UP": up_column,
     "OP": op_column
     }
df_success = pd.DataFrame(data=d)

MCDLSTM
HBNN
FLBNN
LSTMQ
LSTM


In [253]:
# sr_value = sum(df_results.ub_95>df_results.true_gpu)/df_results.shape[0]
# print("SR", sr_value)

# up_value = sum((df_results.ub_95<df_results.true_norm_gpu)*(df_results.true_norm_gpu-df_results.ub_95))/sum((df_results.ub_95<df_results.true_norm_gpu)*df_results.true_norm_gpu)
# up_value = sum((df_results.ub_95<df_results.true_norm_gpu)*(df_results.true_norm_gpu-df_results.ub_95))/sum(df_results.ub_95<df_results.true_norm_gpu)

# up_value = sum((df_results.ub_95<df_results.true_norm_gpu)*((df_results.true_norm_gpu-df_results.ub_95)))/sum(df_results.true_norm_gpu)
# print("UP", up_value)

# op_value = sum((df_results.ub_95>df_results.true_norm_gpu)*(df_results.ub_95-df_results.true_norm_gpu))/sum((df_results.ub_95>df_results.true_norm_gpu)*df_results.true_norm_gpu)

# op_value = sum((df_results.ub_95>df_results.true_norm_gpu)*(df_results.ub_95-df_results.true_norm_gpu))/sum(df_results.ub_95>df_results.true_norm_gpu)

# op_value = sum((df_results.ub_95>df_results.true_norm_gpu)*((df_results.ub_95-df_results.true_norm_gpu)))/sum(df_results.true_norm_gpu)
# print("OP", op_value)

In [254]:
df_accuracy.to_csv("accuracy_table.csv")
df_success.to_csv("success_table.csv")

In [255]:
hbnn_results.columns

Index(['model', 'pred_norm_gpu', 'pred_std', 'true_norm_gpu', 'true_gpu',
       'true_n_gpu', 'pred_gpu', 'ub_90_norm', 'ub_90', 'pred_n_gpu_90',
       'ub_91_norm', 'ub_91', 'pred_n_gpu_91', 'ub_92_norm', 'ub_92',
       'pred_n_gpu_92', 'ub_93_norm', 'ub_93', 'pred_n_gpu_93', 'ub_94_norm',
       'ub_94', 'pred_n_gpu_94', 'ub_95_norm', 'ub_95', 'pred_n_gpu_95',
       'ub_96_norm', 'ub_96', 'pred_n_gpu_96', 'ub_97_norm', 'ub_97',
       'pred_n_gpu_97', 'ub_98_norm', 'ub_98', 'pred_n_gpu_98', 'ub_99_norm',
       'ub_99', 'pred_n_gpu_99'],
      dtype='object')

<h2>Baselines</h2>

a. Running exactly the required number of GPUs that would be specified by our schedulers acting as an oracle. Will have minimal energy use, and a 100% success rate.

b. Always running the maximum number of GPUs acting as a dummy predictor. Will have maximal energy use, and a 100% success rate.

c. Always running the GPUs that were specified by the oracle for the previous time window.

In [34]:
# define the costs (expressed in Watt)
unit_cost = 250
cost_on_5_mins = unit_cost/12
turn_on_cost = 0.20*cost_on_5_mins

n_total_gpus = 6742 # total number of GPUs

<h2>Energy scenario 1</h2>

All GPU machines have the same computational power and they consume the same amount of energy.

The scenario has no memory (no GPU state transitions), i.e. the energy consumption is calculated independently from the previous GPUs states

In [259]:
energy_baseline_a = hbnn_results["true_n_gpu"].values.sum()*cost_on_5_mins*turn_on_cost
energy_baseline_b = n_total_gpus*len(hbnn_results)*cost_on_5_mins*turn_on_cost
baseline_c_values = list(hbnn_results["true_n_gpu"].values[:-1])
baseline_c_values.insert(baseline_c_values[0], 0) # because I do not know the first value
energy_baseline_c = np.sum(baseline_c_values)*cost_on_5_mins*turn_on_cost

In [261]:
models = {"MCD": monte_results,
          "HBNN": hbnn_results,
          "HBNN++": flbnn_results,
          "LSTMQ": lstmq_results,
          # "LSTM": lstm_results
          }
tot_energy_column = []
target_column = []
savings_column = []
model_column = []
for target in target_qos:
    target_column.append(target)
    tot_energy_column.append(energy_baseline_a)
    model_column.append("baseline_a")
    savings_column.append(100 - round(energy_baseline_a/energy_baseline_b*100, 2))

    target_column.append(target)
    tot_energy_column.append(energy_baseline_b)
    model_column.append("baseline_b")
    savings_column.append(100 - round(energy_baseline_b/energy_baseline_b*100, 2))

    target_column.append(target)
    tot_energy_column.append(energy_baseline_c)
    model_column.append("baseline_c")
    savings_column.append(100 - round(energy_baseline_c/energy_baseline_b*100, 2))

    for model in models:
        target_column.append(target)
        model_column.append(model)

        en_value = models[model][f"pred_n_gpu_{target}"].values.sum()*cost_on_5_mins*turn_on_cost
        tot_energy_column.append(en_value)
        savings_column.append(100 - round(en_value/energy_baseline_b*100, 2))

d = {"qos": target_column,
     "model": model_column,
     "tot_energy": tot_energy_column,
     "% energy savings": savings_column
     }
df_scenario_1 = pd.DataFrame(data=d)
df_scenario_1.to_csv("scenario_1.csv")

<h2>Energy scenario 3</h2>

All GPU machines have the same computational power and they consume the same amount of energy when ON.
We model also the state of a GPU {ON, OFF}, and there is a fixed cost on transitioning from OFF to ON. No cost from ON to OFF.

Initial scenario: All GPUs needed are ON, and there is no associated cost for this. Then, GPUs are switched ON and OFF based on the predicted workload. The energy consumtpion is calculated accordingly.

In [262]:
def calculate_consumption(n_gpu_predicted, history, fixed_cost_run, fixed_cost_switch):
    if n_gpu_predicted > history:
        transition_cost = (n_gpu_predicted - history)*fixed_cost_switch
    else:
        transition_cost = 0
    return transition_cost + n_gpu_predicted*fixed_cost_run

In [263]:
models = {"MCD": monte_results,
          "HBNN": hbnn_results,
          "HBNN++": flbnn_results,
          "LSTMQ": lstmq_results,
          # "LSTM": lstm_results
          }

In [264]:
total_cost = 0
gpu_history = hbnn_results["true_n_gpu"].values
for i, n_gpu in enumerate(gpu_history):
    if i == 0:
        total_cost += n_gpu
        continue
    total_cost += calculate_consumption(n_gpu, gpu_history[i-1], cost_on_5_mins, turn_on_cost)
energy_baseline_a = total_cost
energy_baseline_b = n_total_gpus*len(hbnn_results)*cost_on_5_mins + turn_on_cost*n_total_gpus

In [265]:
# TODO baseline c

In [266]:
tot_energy_column = []
target_column = []
savings_column = []
model_column = []
for target in target_qos:
    target_column.append(target)
    tot_energy_column.append(energy_baseline_a)
    model_column.append("baseline_a")
    savings_column.append(100 - round(energy_baseline_a/energy_baseline_b*100, 2))

    target_column.append(target)
    tot_energy_column.append(energy_baseline_b)
    model_column.append("baseline_b")
    savings_column.append(100 - round(energy_baseline_b/energy_baseline_b*100, 2))

    for model in models:
        target_column.append(target)
        model_column.append(model)

        total_cost = 0
        gpu_history = models[model][f"pred_n_gpu_{target}"].values
        for i, n_gpu in enumerate(gpu_history):
            if i == 0:
                total_cost += n_gpu*cost_on_5_mins
                continue
            total_cost += calculate_consumption(n_gpu, gpu_history[i-1], cost_on_5_mins, turn_on_cost)
        tot_energy_column.append(total_cost)
        savings_column.append(100 - round(total_cost/energy_baseline_b*100, 2))

d = {"qos": target_column,
     "model": model_column,
     "tot_energy": tot_energy_column,
     "% energy savings": savings_column
     }
df_scenario_3 = pd.DataFrame(data=d)
df_scenario_3.to_csv("scenario_3.csv")

<h2>Energy scenario 2</h2>

All GPUs are of different characteristics. At the beginning of each time window, all GPUs are OFF.

In [274]:
gpus_specs = {"P100": (1596, float(scaler.inverse_transform(np.array(0.000404575892857).reshape(-1,1))), 0.0208),
              "T4": (994, float(scaler.inverse_transform(np.array(0.00056640625).reshape(-1,1))), 0.0058),
              "V100": (1912, float(scaler.inverse_transform(np.array(0.000283203125).reshape(-1,1))), 0.0233),
              "MISC": (2240, float(scaler.inverse_transform(np.array(0.000361596009975).reshape(-1,1))), 0.0208),
              }
for spec in gpus_specs:
    print(f"{spec}: {gpus_specs[spec]}")

P100: (1596, 10659485.301714368, 0.0208)
T4: (994, 10662963.714422604, 0.0058)
V100: (1912, 10656876.492183197, 0.0233)
MISC: (2240, 10658561.4838754, 0.0208)


In [278]:
float(scaler.inverse_transform(np.array(1.45).reshape(-1,1)))

41817367.13571128

In [119]:
def allocate_resources(w_demand: float,
                       df_stat: pd.DataFrame
                       ) -> float:
    df_allocation = df_stat.copy()
    wd_rem = w_demand
    cost = 0
    while wd_rem > 0:
        # if a single GPU can satisfy the remaining work, we choose the cheapest one.
        if wd_rem < max(df_allocation.w):
            df_allocation = df_allocation[df_allocation["w"] > wd_rem]
            df_allocation.sort_values(by='cost', inplace=True)
            cost += df_allocation.cost.head(1).values[0]
            break

        # Otherwise we select the most efficient
        wd_rem -= df_allocation.w.tail(1).values[0]
        cost += df_allocation.cost.tail(1).values[0]
        # remove the GPU used
        df_allocation.drop(df_allocation.tail(1).index,inplace=True) # drop last n rows
    return cost

In [141]:
# build dataframe of GPUs
w = [] # stores workload that GPUs can provide
e = [] # stores energy consumption
c = [] # stores energy cost to turn on GPUs
s = [] # stores the status of GPUs
g_name = [] # stores the name of the GPUs
for gpu_name in gpus_specs:
    w_value = gpus_specs[gpu_name][1]
    n_g = gpus_specs[gpu_name][0]
    e_g = gpus_specs[gpu_name][2]
    c_g = gpus_specs[gpu_name][2]*0.2
    g_name.extend([gpu_name for i in range(n_g)])
    w.extend([w_value for i in range(n_g)])
    e.extend([e_g for i in range(n_g)])
    c.extend([c_g for i in range(n_g)])
    s.extend([1 for i in range(n_g)])

d = {"gpu_name": g_name, 'w': w, 'e': e, 'c': c, 's': s}
df_gpus_status = pd.DataFrame(data=d)

# efficiency of GPU = workload provided / energy required
df_gpus_status['cost'] = df_gpus_status.e + df_gpus_status.s * df_gpus_status.c
df_gpus_status['eff'] = df_gpus_status.w/ df_gpus_status.cost

df_gpus_status = df_gpus_status.sort_values(by='eff')
print(df_gpus_status.head(2))

     gpu_name         w       e         c  s      cost       eff
3370     V100  0.000283  0.0233  0.005825  1  0.029125  0.009724
2868     V100  0.000283  0.0233  0.005825  1  0.029125  0.009724


In [142]:
# pred_workloads = {"baseline_a": list(hbnn_results["true_norm_gpu"].values),
#                   "HBNN": list(hbnn_results["ub_95"].values),
#                   }

pred_workloads = {"baseline_a": hbnn_results["true_norm_gpu"].values[:50],
                  "baseline_b": [1.45 for i in range(len(hbnn_results))], # max possible workload
                  "HBNN": hbnn_results["ub_95"].values[:50],
                  "MCD": monte_results["ub_95"].values[:50],
                  "HBNN++": flbnn_results["ub_95"].values[:50],
                  "LSTMQ": lstmq_results["ub_95"].values[:50],
                  "LSTM": lstm_results["ub_95"].values[:50],
                  }
print(f"{datetime.datetime.now()} -- BEGIN!")
scenario_2_costs = {}
for model in pred_workloads:
    demands = pred_workloads[model]
    tot_costs = [allocate_resources(demand, df_gpus_status) for demand in demands]
    scenario_2_costs[model] = np.sum(tot_costs)
    print(f"{datetime.datetime.now()} -- Done with model {model}!")
print(f"{datetime.datetime.now()} -- END!")

2023-06-23 17:26:25.239993 -- BEGIN!
2023-06-23 17:27:57.800369 -- Done with model baseline_a!


KeyboardInterrupt: 

In [ ]:
model_column = [model for model in scenario_2_costs]
energy_value = [100 - round(scenario_2_costs[model]/scenario_2_costs["baseline_b"]*100, 2) for model in scenario_2_costs]
d = {"model": model_column, "% energy savings": energy_value}
scenario_2_costs = pd.DataFrame(data=d)
scenario_2_costs

In [123]:
scenario_2_costs

{'baseline_a': 2317.0266000000797}

In [61]:
hbnn_results.head(1)

,model,pred_gpu,pred_std,true_norm_gpu,true_gpu,true_n_gpu,ub_95,pred_n_gpu_95
0,HBNN,0.79237,0.058169,0.804885,2.918980e+07,3718,0.88805,4103
